# Data Cleaning Listado de Productores Autorizados

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
# Definición de funciones

def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252')

Lectura del listado de productores autorizados

In [4]:
listado_productores = load_datasets('../../data/productores_autorizados/')

# 2. Limpieza de los datos

## 2.1 INEGI

In [5]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [6]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2
2,10010096,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.883756,-102.357122,1861,F13D18,41,24,17,12
3,10010100,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.854599,-102.372745,1879,F13D18,0,0,0,0
4,10010102,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.780180,-102.357295,1861,F13D18,8,*,*,2


In [7]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [8]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT     int64
NOM_ENT    object
CVE_MUN     int64
NOM_MUN    object
dtype: object

In [9]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

Shape of dataset_inegi:  (299568, 4)
Shape of dataset_inegi_clean:  (2476, 4)


A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [10]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
0,1,Aguascalientes,1,Aguascalientes
708,1,Aguascalientes,2,Asientos
945,1,Aguascalientes,3,Calvillo
1237,1,Aguascalientes,4,Cosío
1330,1,Aguascalientes,5,Jesús María


In [11]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
299150,32,Zacatecas,54,Villa Hidalgo
299211,32,Zacatecas,55,Villanueva
299363,32,Zacatecas,56,Zacatecas
299484,32,Zacatecas,57,Trancoso
299526,32,Zacatecas,58,Santa María de la Paz


In [12]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2476 entries, 0 to 299526
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CVE_ENT  2476 non-null   int64 
 1   NOM_ENT  2476 non-null   object
 2   CVE_MUN  2476 non-null   int64 
 3   NOM_MUN  2476 non-null   object
dtypes: int64(2), object(2)
memory usage: 96.7+ KB


In [13]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

Los valores únicos en cada columna son:
 CVE_ENT      32
NOM_ENT      32
CVE_MUN     570
NOM_MUN    2332
dtype: int64


In [14]:
# Creamos una columna con la clave única por municipio

dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[
    'CVE_MUN'].astype(str)

dataset_inegi_clean.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/2609109509.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[


,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
0,1,Aguascalientes,1,Aguascalientes,1-1
708,1,Aguascalientes,2,Asientos,1-2
945,1,Aguascalientes,3,Calvillo,1-3
1237,1,Aguascalientes,4,Cosío,1-4
1330,1,Aguascalientes,5,Jesús María,1-5


In [15]:
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
299150,32,Zacatecas,54,Villa Hidalgo,32-54
299211,32,Zacatecas,55,Villanueva,32-55
299363,32,Zacatecas,56,Zacatecas,32-56
299484,32,Zacatecas,57,Trancoso,32-57
299526,32,Zacatecas,58,Santa María de la Paz,32-58


### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [16]:
# Estandarizamos la limpieza de los datos
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/1220250931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/1220250931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)


In [17]:
dataset_inegi_clean

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique,NOM_ENT_Clean,NOM_MUN_Clean
0,1,Aguascalientes,1,Aguascalientes,1-1,aguascalientes,aguascalientes
708,1,Aguascalientes,2,Asientos,1-2,aguascalientes,asientos
945,1,Aguascalientes,3,Calvillo,1-3,aguascalientes,calvillo
1237,1,Aguascalientes,4,Cosío,1-4,aguascalientes,cosio
1330,1,Aguascalientes,5,Jesús María,1-5,aguascalientes,jesus maria
...,...,...,...,...,...,...,...
299150,32,Zacatecas,54,Villa Hidalgo,32-54,zacatecas,villa hidalgo
299211,32,Zacatecas,55,Villanueva,32-55,zacatecas,villanueva
299363,32,Zacatecas,56,Zacatecas,32-56,zacatecas,zacatecas
299484,32,Zacatecas,57,Trancoso,32-57,zacatecas,trancoso


In [18]:
dataset_inegi_clean.shape

(2476, 7)

## 2.2 Listado de Productores Autorizados

In [19]:
listado_productores.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
       'NOMBRE (S)', 'PAQUETE'],
      dtype='object')

In [20]:
listado_productores.head()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,TLAXCALA,ACUAMANALA DE MIGUEL HIDALGO,23-PRONAFE-FERT-000018-S000-TL,AGUILA,LIMA,J. EUGENIO,2
1,TLAXCALA,ACUAMANALA DE MIGUEL HIDALGO,23-PRONAFE-FERT-000016-S000-TL,CORTE,ESPINOZA,MA. PETRA APOLONIA,2
2,TLAXCALA,ACUAMANALA DE MIGUEL HIDALGO,23-PRONAFE-FERT-000005-S000-TL,CORTE,PICHON,TERESA,2
3,TLAXCALA,ACUAMANALA DE MIGUEL HIDALGO,23-PRONAFE-FERT-000006-S000-TL,CUAHTEPITZI,LUNA,FLORENCIO,2
4,TLAXCALA,ACUAMANALA DE MIGUEL HIDALGO,23-PRONAFE-FERT-000007-S000-TL,CUAHTEPITZI,SANCHEZ,JOSE SIMON MARTIN,2


In [21]:
# Mostramos el tipo de dato de cada columna
listado_productores.dtypes
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


In [22]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
COLUMNS_TO_VIEW = ['ESTADO', 'MUNICIPIO', 'PAQUETE']
for column in COLUMNS_TO_VIEW:
    unique_values = listado_productores[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\n{column}\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)


ESTADO
Cantidad de valores unicos 20. Valores únicos en la columna ESTADO: ['AGUASCALIENTES' 'CAMPECHE' 'CHIAPAS' 'CIUDAD DE MEXICO' 'COLIMA'
 'DURANGO' 'GUERRERO' 'HIDALGO' 'JALISCO' 'MEXICO' 'MICHOACAN DE OCAMPO'
 'MORELOS' 'MÉXICO' 'OAXACA' 'PUEBLA' 'QUERETARO DE ARTEAGA'
 'QUINTANA ROO' 'TLAXCALA' 'VERACRUZ DE IGNACIO DE LA LLAVE' 'YUCATAN']

Cantidad de valores únicos en MUNICIPIO: 1695
Valores únicos en la columna MUNICIPIO:
ABALA
ABEJONES
ACACOYAGUA
ACAJETE
ACALA
ACAMBAY
ACANCEH
ACAPETAHUA
ACAPULCO DE JUAREZ
ACATENO
ACATEPEC
ACATIC
ACATLAN
ACATLAN DE JUAREZ
ACATLAN DE PEREZ FIGUEROA
ACATZINGO
ACAXOCHITLAN
ACOLMAN
ACTEOPAN
ACTOPAN
ACUAMANALA DE MIGUEL HIDALGO
ACUITZIO
ACULCO
AGUA BLANCA DE ITURBIDE
AGUASCALIENTES
AGUILILLA
AHUACATLAN
AHUACUOTZINGO
AHUALULCO DE MERCADO
AHUATLAN
AHUAZOTEPEC
AHUEHUETITLA
AJACUBA
AJALPAN
AJUCHITLAN DEL PROGRESO
AKIL
ALBINO ZERTUCHE
ALCOZAUCA DE GUERRERO
ALDAMA
ALFAJAYUCAN
ALJOJUCA
ALMOLOYA
ALMOLOYA DE ALQUISIRAS
ALMOLOYA DE JUAREZ
ALMOLOYA DEL RIO
A

In [23]:
municipio_counts = listado_productores['MUNICIPIO'].value_counts().reset_index()
municipio_counts.columns = ['MUNICIPIO', 'COUNT']
print(municipio_counts)

                 MUNICIPIO  COUNT
0           LAS MARGARITAS  19497
1      VENUSTIANO CARRANZA  16583
2       CHILAPA DE ALVAREZ  16484
3                 ACATEPEC  13163
4     AYUTLA DE LOS LIBRES  13160
...                    ...    ...
1689  SAN ANDRES TEPETLAPA      1
1690            IZTAPALAPA      1
1691              PROGRESO      1
1692              CELESTUN      1
1693   SAN JACINTO AMILPAS      1

[1694 rows x 2 columns]


¿Qué ESTADOS tienen algún municipio NULL?

In [24]:
municipios_null = listado_productores[listado_productores['MUNICIPIO'].isnull()]

# ¿Qué ESTADOS tienen algún municipio Null?
print(municipios_null['ESTADO'].unique())

['VERACRUZ DE IGNACIO DE LA LLAVE']


In [25]:
# Filas duplicadas y filas únicas
duplicated_rows_productores = listado_productores.duplicated()
number_of_duplicated_rows_productores = duplicated_rows_productores.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows_productores}")
print(f"El número de filas únicas es: {listado_productores.shape[0] - number_of_duplicated_rows_productores}")

El número de filas duplicadas es: 0
El número de filas únicas es: 1525720


No hay filas completas duplicadas.

In [26]:
# Valores nulos
listado_productores.isna().sum()

ESTADO                  0
MUNICIPIO           75512
ACUSE                   0
APELLIDO PATERNO     4294
APELLIDO MATERNO    28213
NOMBRE (S)           4293
PAQUETE                 0
dtype: int64

Existen 75512 filas con el municipio nulo. 
Y muchas filas con nombre y apellidos vacíos.


In [27]:
# Analicemos dónde los municipios están vacíos
listado_productores[(listado_productores['MUNICIPIO'].isna())]


,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
750919,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000001-S000-VZ,ABURTO,ALARCON,ASCENCION,1
750920,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000002-S000-VZ,ABURTO,ALARCON,GENARO,1
750921,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000003-S000-VZ,ABURTO,MARTINEZ,MARIA DEL CARMEN,2
750922,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000004-S000-VZ,ABURTO,MORALES,MIGUEL ANGEL,2
750923,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000005-S000-VZ,ALARCON,GUZMAN,MARCIANO,2
...,...,...,...,...,...,...,...
826426,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081776-S000-VZ,REYES,HERNANDEZ,RUTILIO,1
826427,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081842-S000-VZ,SANTIAGO,SARMIENTO,LEOBARDO,2
826428,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081843-S000-VZ,VAZQUEZ,GAONA,JOSE,2
826429,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081845-S000-VZ,VEGA,RODRIGUEZ,JUAN,1


Todas las filas con municipios vacíos vienen del estado de Veracruz de Ignacio de la Llave.

In [28]:
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los nombres de municipios en un dataset más ligero llamado **Estados_productores**.

In [29]:
# Seleccionar solo las dos primeras columnas
Estados_productores = listado_productores[['ESTADO', 'MUNICIPIO']]

# Obtener las filas únicas
Estados_productores = Estados_productores.drop_duplicates()

In [30]:
Estados_productores.shape

(1862, 2)

In [31]:
# Estandarizamos la limpieza de los datos
Estados_productores['ESTADO_Clean'] = Estados_productores['ESTADO'].apply(clean_text)
Estados_productores['MUNICIPIO_Clean'] = Estados_productores['MUNICIPIO'].apply(clean_text)

In [32]:
Estados_productores[Estados_productores['MUNICIPIO_Clean'] == 'san juan mixtepec -dto. 26 -']

,ESTADO,MUNICIPIO,ESTADO_Clean,MUNICIPIO_Clean


Creamos columna clave

In [33]:
# Primero creemos una columna clave en cada dataset -> Estados productores

Estados_productores["ESTADO_Clean"] = Estados_productores["ESTADO_Clean"].astype(str)
Estados_productores["MUNICIPIO_Clean"] = Estados_productores["MUNICIPIO_Clean"].astype(str)

Estados_productores["KEY_prod"] = Estados_productores["ESTADO_Clean"] + "-" + Estados_productores["MUNICIPIO_Clean"]

In [34]:
Estados_productores.shape

(1862, 5)

# 3. Diccionario de los datasets de INEGI Y PRODUCTORES AUTORIZADOS

A este punto llegamos con 3 datasets:  
    1. dataset_inegi_clean: municipios y estados de INEGI con los nombres estandarizados  
    2. Productores_Autorizados: listado de productores autorizados con municipios y estados estandarizados  
    3. Estados_productores: Municipios y Estados únicos que aparecen en el listado de Productores Autorizados, sin el formato INEGI.  

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Estados_productores. Para ello haremos un Left join entre Estados_productores y dataset_inegi_clean.

### 3.1 Left join

In [35]:
# Primero creemos una columna clave en cada dataset -> INEGI

dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY_inegi"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/1010743891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/1010743891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/1010743891.py:6

In [36]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2476 entries, 0 to 299526
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CVE_ENT         2476 non-null   int64 
 1   NOM_ENT         2476 non-null   object
 2   CVE_MUN         2476 non-null   int64 
 3   NOM_MUN         2476 non-null   object
 4   CVE_MUN_Unique  2476 non-null   object
 5   NOM_ENT_Clean   2476 non-null   object
 6   NOM_MUN_Clean   2476 non-null   object
 7   KEY_inegi       2476 non-null   object
dtypes: int64(2), object(6)
memory usage: 174.1+ KB


In [37]:
Estados_productores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1862 entries, 0 to 1525005
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ESTADO           1862 non-null   object
 1   MUNICIPIO        1861 non-null   object
 2   ESTADO_Clean     1862 non-null   object
 3   MUNICIPIO_Clean  1862 non-null   object
 4   KEY_prod         1862 non-null   object
dtypes: object(5)
memory usage: 87.3+ KB


In [38]:
Estados_productores = Estados_productores.drop(['ESTADO', 'MUNICIPIO'], axis=1)
Estados_productores = Estados_productores.drop_duplicates()
Estados_productores.shape

(1736, 3)

Comprobamos que la columna KEY es realmente clave.

In [39]:
Estados_productores['KEY_prod'].nunique()

1736

In [40]:

# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge(df_inegi, df_prod, key1, key2, threshold=96, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='inner', 
                         suffixes=('_inegi', '_prod'))

    return df_merged


In [41]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge(dataset_inegi_clean, Estados_productores, 'KEY_inegi', 'KEY_prod')

# Mostrar el resultado
diccionario.shape
#print(diccionario.head())

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/413782971.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_39653/413782971.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['match_score'] = [match[1] if match else None for match in matches]


(1705, 13)

In [42]:
diccionario_counts = diccionario['KEY_inegi'].value_counts().reset_index()
diccionario_counts.columns = ['KEY_inegi', 'COUNT']
print(diccionario_counts)

                           KEY_inegi  COUNT
0           oaxaca-san juan mixtepec      2
1          oaxaca-san pedro mixtepec      2
2            oaxaca-santa cruz xitla      1
3          oaxaca-santa maria apazco      1
4        oaxaca-santa maria alotepec      1
...                              ...    ...
1698                 mexico-sultepec      1
1699  mexico-soyaniquilpan de juarez      1
1700              mexico-santo tomas      1
1701    mexico-san simon de guerrero      1
1702                  yucatan-yobain      1

[1703 rows x 2 columns]


In [43]:
diccionario.to_csv('../../data/merged_dataset.csv', index=False)

## 3.2 Productores Autorizados

Esta sección se encarga de completar el listado original de Productores Autorizados con los nombre corregido de INEGI usando el diccionario.

In [44]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_productores['ESTADO_Clean'] = listado_productores['ESTADO'].apply(clean_text)
listado_productores['MUNICIPIO_Clean'] = listado_productores['MUNICIPIO'].apply(clean_text)
listado_productores['Estado-mun-KEY'] = listado_productores['ESTADO_Clean'].astype(str) + '-' + listado_productores['MUNICIPIO_Clean'].astype(str)

diccionario_Sin_VC = diccionario[diccionario["NOM_ENT"] != "Veracruz de Ignacio de la Llave"]

In [45]:
listado_productores.shape

(1525720, 10)

In [46]:
# Hacer el join
listado_productores_complete = pd.merge(listado_productores, diccionario_Sin_VC, left_on="Estado-mun-KEY",
                                        right_on="KEY_prod", how='left', suffixes=('_prod', '_inegi'))

In [47]:
listado_productores_complete[['CVE_ENT', 'CVE_MUN']] = listado_productores_complete['CVE_MUN_Unique'].str.split('-', expand=True)


In [48]:
listado_productores_complete.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
       'NOMBRE (S)', 'PAQUETE', 'ESTADO_Clean_prod', 'MUNICIPIO_Clean_prod',
       'Estado-mun-KEY', 'CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN',
       'CVE_MUN_Unique', 'NOM_ENT_Clean', 'NOM_MUN_Clean', 'KEY_inegi',
       'best_match', 'match_score', 'ESTADO_Clean_inegi',
       'MUNICIPIO_Clean_inegi', 'KEY_prod'],
      dtype='object')

In [49]:
# Seleccionamos las columnas que nos interesan
listado_productores_complete = listado_productores_complete[['ESTADO', 'MUNICIPIO','NOM_MUN','NOM_ENT','CVE_ENT','CVE_MUN','ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'KEY_inegi']]

In [50]:
# Revisamos el dataset
print(listado_productores_complete.shape)
print(listado_productores_complete.columns)
print(listado_productores_complete.head())

(1528499, 12)
Index(['ESTADO', 'MUNICIPIO', 'NOM_MUN', 'NOM_ENT', 'CVE_ENT', 'CVE_MUN',
       'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRE (S)',
       'PAQUETE', 'KEY_inegi'],
      dtype='object')
     ESTADO                     MUNICIPIO                       NOM_MUN  \
0  TLAXCALA  ACUAMANALA DE MIGUEL HIDALGO  Acuamanala de Miguel Hidalgo   
1  TLAXCALA  ACUAMANALA DE MIGUEL HIDALGO  Acuamanala de Miguel Hidalgo   
2  TLAXCALA  ACUAMANALA DE MIGUEL HIDALGO  Acuamanala de Miguel Hidalgo   
3  TLAXCALA  ACUAMANALA DE MIGUEL HIDALGO  Acuamanala de Miguel Hidalgo   
4  TLAXCALA  ACUAMANALA DE MIGUEL HIDALGO  Acuamanala de Miguel Hidalgo   

    NOM_ENT CVE_ENT CVE_MUN                           ACUSE APELLIDO PATERNO  \
0  Tlaxcala      29      22  23-PRONAFE-FERT-000018-S000-TL           AGUILA   
1  Tlaxcala      29      22  23-PRONAFE-FERT-000016-S000-TL            CORTE   
2  Tlaxcala      29      22  23-PRONAFE-FERT-000005-S000-TL            CORTE   
3  Tlaxcala      

In [51]:
listado_productores_complete.to_csv('../../data/listado_productores_complete.csv', index=False)

Recursos adicionales; enseñar a Jose Luis y decidir que hacemos.

In [52]:
unique_listado = listado_productores['Estado-mun-KEY'].unique()
unique_diccionario = diccionario_Sin_VC['KEY_prod'].unique()

# To get the common values
common_values = set(unique_listado).intersection(unique_diccionario)
print("Common values:", common_values)

# To get values in listado_productores not in diccionario_Sin_VC
in_listado_not_diccionario = set(unique_listado) - set(unique_diccionario)
print("Values in 'listado_productores' not in 'diccionario_Sin_VC':", in_listado_not_diccionario)

# To get values in diccionario_Sin_VC not in listado_productores
in_diccionario_not_listado = set(unique_diccionario) - set(unique_listado)
print("Values in 'diccionario_Sin_VC' not in 'listado_productores':", in_diccionario_not_listado)

Common values: {'yucatan-tunkas', 'yucatan-izamal', 'jalisco-ameca', 'hidalgo-acatlan', 'puebla-santa isabel cholula', 'oaxaca-cosoltepec', 'oaxaca-chahuites', 'oaxaca-magdalena mixtepec', 'oaxaca-san juan lalana', 'hidalgo-tasquillo', 'mexico-atenco', 'jalisco-bolanos', 'colima-comala', 'puebla-san juan atenco', 'mexico-otzolotepec', 'puebla-jonotla', 'puebla-tecomatlan', 'hidalgo-atotonilco el grande', 'yucatan-uayma', 'mexico-isidro fabela', 'yucatan-dzidzantun', 'puebla-huitzilan de serdan', 'oaxaca-san agustin etla', 'michoacan de ocampo-tzintzuntzan', 'puebla-la magdalena tlatlauquitepec', 'jalisco-ayotlan', 'oaxaca-santa maria tataltepec', 'chiapas-acacoyagua', 'guerrero-ayutla de los libres', 'campeche-hecelchakan', 'michoacan de ocampo-zacapu', 'michoacan de ocampo-panindicuaro', 'puebla-yehualtepec', 'oaxaca-yutanduchi de guerrero', 'oaxaca-santa catarina juquila', 'oaxaca-heroica ciudad de tlaxiaco', 'oaxaca-martires de tacubaya', 'oaxaca-santa maria tonameca', 'hidalgo-mola

In [53]:
# Create a DataFrame with unique values from 'listado_productores'
df_relations = pd.DataFrame(unique_listado, columns=['Estado-mun-KEY'])

# Check if each value is present in 'diccionario_Sin_VC'
df_relations['In_diccionario_Sin_VC'] = df_relations['Estado-mun-KEY'].isin(unique_diccionario)

# Repeat the process for 'diccionario_Sin_VC'
df_relations_diccionario = pd.DataFrame(unique_diccionario, columns=['KEY_prod'])
df_relations_diccionario['In_listado_productores'] = df_relations_diccionario['KEY_prod'].isin(unique_listado)

# Merge the two DataFrames
df_relations = pd.merge(df_relations, df_relations_diccionario, left_on='Estado-mun-KEY', right_on='KEY_prod', how='outer')

df_relations

,Estado-mun-KEY,In_diccionario_Sin_VC,KEY_prod,In_listado_productores
0,tlaxcala-acuamanala de miguel hidalgo,True,tlaxcala-acuamanala de miguel hidalgo,True
1,tlaxcala-amaxac de guerrero,True,tlaxcala-amaxac de guerrero,True
2,tlaxcala-apetatitlan de antonio carvajal,True,tlaxcala-apetatitlan de antonio carvajal,True
3,tlaxcala-apizaco,True,tlaxcala-apizaco,True
4,tlaxcala-atlangatepec,True,tlaxcala-atlangatepec,True
...,...,...,...,...
1731,puebla-santa catarina tlaltempan,True,puebla-santa catarina tlaltempan,True
1732,puebla-teteles de avila castillo,True,puebla-teteles de avila castillo,True
1733,puebla-tlacuilotepec,True,puebla-tlacuilotepec,True
1734,puebla-xochiltepec,True,puebla-xochiltepec,True


In [54]:
df_relations[(df_relations['In_diccionario_Sin_VC'] == False) | (df_relations['In_listado_productores'] == False)]

,Estado-mun-KEY,In_diccionario_Sin_VC,KEY_prod,In_listado_productores
111,guerrero-jose azueta,False,NaN,NaN
371,chiapas-belisario dominguez,False,NaN,NaN
388,chiapas-cintalapa,False,NaN,NaN
683,jalisco-tlaquepaque,False,NaN,NaN
932,oaxaca-san blas atempa,False,NaN,NaN
1263,oaxaca-santiago chazumba,False,NaN,NaN
1349,oaxaca-tezoatlan de segura y luna,False,NaN,NaN
1360,oaxaca-villa de tututepec de melchor ocampo,False,NaN,NaN
1363,oaxaca-villa hidalgo,False,NaN,NaN
1370,oaxaca-zapotitlan del rio,False,NaN,NaN
